# Keras Modeling 

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 09/12/2024   | Martin | Created   | Created notebook for Chp 3. Sequential model started | 
| 10/12/2024   | Martin | Update   | Completed Sequential and Functional API. Started Subclassing API | 
| 11/12/2024   | Martin | Update   | Completed Subclassing API | 

# Content

* [Introduction](#introduction)
* [Understanding Keras Layers](#understanding-keras-layers)
* [Sequential API](#keras-sequential-api)
* [Functional API](#keras-functional-api)
* [Subclassing API](#keras-subclassing-api)

# Introduction

Keras is a high-level API with multiple ML frameworks as its backend with Tensorflow being one of them. Provides an easy-to-use and accessible library to enable fast experimentation.

Keras is the official high-level API for Tensorflow v2. It integrates TensorFlow-specific functionality like eager execution, data pipelines and Estimators, optimized for the Tensorflow backend

The only difference between the Tensorflow version and the typical Keras package is in how it's imported

# Understanding Keras Layers

Keras layers are the fundamental building blocks of Keras models. Each layer receives data as input, does a specific task and returns an output

* __Core layers__: Dense, Activation, Flatten, Input, ...
* __Convolutional layers__: Conv1D, Conv2D, Cropping2D, ...
* __Pooling layers__: perform a downsampling operation/ MaxPooling1D, AveragePooling2D, ...
* __Recurrent layers__: RNN, SimpleRNN, LSTM, ...
* __Embedding layers__: used as the first layers to create dense vectors of fixed size to represent more complex details (e.g text data)
* __Merge layers__: Add, Subtract, Multiply, ...
* __Advanced activation layers__: LeakyReLU, Softmax, ...
* __Batch normalisation layers__: normalises the activation of the previous layer at each batch
* __Noise layers__: GausianNoise, GausianDropout, AlphaDropout
* __Layer wrappers__: TimeDistributed applies a layer to every temporal slice of an input and bidirectional wrapper for RNNs
* __Locally connected layers__: LocallyConnected1D and LocallyConnected2D
* __Custom layers__: able to write custom layers using the Subclassing API

In [ ]:
# Standard functions used in Keras layers
layer.get_weights() # returns weights of layer as list of NumPy arrays
layer.set_weights(weights) # fixes the weights of the layer 

## For shared layers - layers that are used multiple times in the network
layer.get_input_at(node_index)
layer.get_output_at(node_index)

## Get shape
layer.input_shape
layer.output_shape

## Get a layers configuration, does not include weights or connectivity information
layer.get_config()
layer.from_config(config) # configs are stored in a dictionary

---

# Keras Sequential API

Create sequential models which are linear stacks of layers. The model architecture is specified and the training, tuning and testing loop is built around the model specified.

The Sequential API follows the delayed-build pattern: if no input shape is specified on the first layer, the model gets built the first time the model is called on some input data.

Graph is finalized with the `compile` method which configures the modle before the learning phase. Then the model is evaluated and able to make predictions.

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
import numpy as np

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "2"

In [5]:
# Create the Sequential model

## model here is a categorical classifier for 10 different categories
model = tf.keras.Sequential([
  tf.keras.layers.Dense(1024, input_dim=64), # first number represents number of nodes
  tf.keras.layers.Activation('tanh'),
  tf.keras.layers.Dense(256),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Activation('softmax')
])

In [6]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 1024)           │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │         2,570 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 331,530 (1.26 MB)

 Trainable params: 331,530 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Another method of declaring the model is to use the .add() method
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024, input_dim=64))
model.add(tf.keras.layers.Activation('tanh'))
# ...
model.add(tf.keras.layers.Activation('softmax'))

## Layer configurations

In [ ]:
# Layers can have different parameters to specify their functions

## specifies the number of inputs the layer expects to receive
Dense(256, input_dim=64)

## specifies the activation function of this layer
Dense(256, activation='sigmoid') 

## specifies the initialisation strategy for weights and biases
Dense(256, kernel_initializer='random_normal') 
Dense(256, bias_initializer=tf.keras.initializers.Constant(value=5))

## regularizers for kernel and bias
Dense(256, kernel_regularizer=tf.keras.regularizers.l1(0.01))
Dense(256, bias_regularizer=tf.keras.regularizers.l2(0.01))

In [ ]:
# Model compilation - specifies details abot the training mechanism
model.compile(
  optimizer="adam", # optimisation algorithm
  loss="categorical_crossentropy", # loss function can be custom - return a scalar of loss for each data point
  metrics=["accuracy"] # metrics to judge the model performance - not used in training process
  # use run_eagerly to evaluate eagerly
)

In [ ]:
# Model fit - training process begins
model.fit(
  data,
  labels,
  epochs=10, # number of iterations over the entire input data
  batch_size=50, # amount of data to process per batch
  validation_data=(val_data, val_labels) # validation data - monitor performance of model
)

Layer configuration recommendations:

1. Always set the __input shape__ for the first layer - shape must be the same as the training data. Subsequent layers can perform inference.
2. Keras is defined to support any batch size, so only the number of features is needed to be specified. (but can be controlled using `batch_size` parameter)

If the input shape is not specified, no methods can be called on the layer

## Example model

In [3]:
# Generate data
data = np.random.random((2000, 64)) # 2000 entries, 64 features
labels = np.random.random(( 2000, 10 ))
val_data = np.random.random(( 500, 64 ))
val_labels = np.random.random(( 500, 10 ))
test_data = np.random.random(( 500, 64 ))
test_labels = np.random.random(( 500, 10 ))

In [5]:
# Define model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(1024, input_dim=64), # first number represents number of nodes
  tf.keras.layers.Activation('tanh'),
  tf.keras.layers.Dense(256),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Activation('softmax')
])

In [6]:
# Define model training parameters
model.compile(
  optimizer="adam", # optimisation algorithm
  loss="categorical_crossentropy", # loss function can be custom - return a scalar of loss for each data point
  metrics=["accuracy"] # metrics to judge the model performance - not used in training process
  # use run_eagerly to evaluate eagerly
)

In [7]:
# Train the model
model.fit(
  data,
  labels,
  epochs=10, # number of iterations over the entire input data
  batch_size=50, # amount of data to process per batch
  validation_data=(val_data, val_labels) # validation data - monitor performance of model
)

Epoch 1/10


I0000 00:00:1733797392.590412     110 service.cc:146] XLA service 0x7fb860003c70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733797392.590457     110 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-12-10 02:23:12.621860: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-10 02:23:12.727804: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
2024-12-10 02:23:13.583474: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_145', 20 bytes spill stores, 20 bytes spill loads

2024-12-10 02:23:14.249693: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in functio

 1/40 ━━━━━━━━━━━━━━━━━━━━ 2:02 3s/step - accuracy: 0.1200 - loss: 11.2705

I0000 00:00:1733797395.266738     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0993 - loss: 23.0681 - val_accuracy: 0.0920 - val_loss: 107.6519
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1017 - loss: 145.7135 - val_accuracy: 0.1100 - val_loss: 243.0333
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1044 - loss: 240.1554 - val_accuracy: 0.1000 - val_loss: 107.3819
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0834 - loss: 83.5420 - val_accuracy: 0.1000 - val_loss: 29.9293
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0907 - loss: 47.6700 - val_accuracy: 0.1180 - val_loss: 73.9437
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0957 - loss: 77.8198 - val_accuracy: 0.1180 - val_loss: 73.6665
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1102 - loss: 94.1683 - val_accuracy: 0.1180 - val_loss: 99.7233
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0974 - loss: 117.4692 - val_accuracy: 0.1000 

In [9]:
# Check the models performance on training data
model.evaluate(data, labels, batch_size=50)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1004 - loss: 174.2154


[173.08287048339844, 0.10100000351667404]

In [10]:
# Get results of model on test set
model.predict(test_data, batch_size=50)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


array([[3.68650632e-26, 1.08123525e-13, 9.91296702e-07, ...,
        1.49798806e-42, 9.99966741e-01, 1.29173701e-12],
       [3.68650632e-26, 1.08123525e-13, 9.91296702e-07, ...,
        1.49798806e-42, 9.99966741e-01, 1.29173701e-12],
       [3.68650632e-26, 1.08123525e-13, 9.91296702e-07, ...,
        1.49798806e-42, 9.99966741e-01, 1.29173701e-12],
       ...,
       [3.68650632e-26, 1.08123525e-13, 9.91296702e-07, ...,
        1.49798806e-42, 9.99966741e-01, 1.29173701e-12],
       [3.68650632e-26, 1.08123525e-13, 9.91296702e-07, ...,
        1.49798806e-42, 9.99966741e-01, 1.29173701e-12],
       [3.68650632e-26, 1.08123525e-13, 9.91296702e-07, ...,
        1.49798806e-42, 9.99966741e-01, 1.29173701e-12]], dtype=float32)

---

# Keras Functional API

Keras Sequential API is limited to a liner topology (layers can only follow one after the other). The Functional API allows defining more complex models with a non-linear topology (e.g ResNet, Inception, etc.)

Multiple inputs, Multiple outputs, residual connections with non-sequential flow, and shared and reusable layers.

The Functional API is a way to build a graph of layers and create more flexible models.

__How it works__

The Functional API works by individually defining the layers and then passing the outputs of one layer to another at each step. Like drawing the arrows between the layers manually. More time consuming but also __more configurable__ in terms of layer architecture design.

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, TimeDistributed
import keras.models
import numpy as np

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "2"

Working of the MNIST dataset

In [12]:
# Load data
mnist = tf.keras.datasets.mnist
(X_mnist_train, y_mnist_train), (X_mnist_test, y_mnist_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [13]:
# Create the input node
inputs = tf.keras.Input(shape=(28, 28))

# Flatten the image
flatten_layer = keras.layers.Flatten()
flatten_output = flatten_layer(inputs) # a layer call action - "passing" the output of inputs layer into flatten layer

# Dense layer
dense_layer = tf.keras.layers.Dense(50, activation='relu')
dense_output = dense_layer(flatten_output)

# Output layer
predictions = tf.keras.layers.Dense(10, activation='softmax')(dense_output)

# Define the mfinal model
model = keras.Model(inputs=inputs, outputs=predictions)

In [14]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 28, 28)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 50)             │        39,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,760 (155.31 KB)

 Trainable params: 39,760 (155.31 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(
  optimizer='sgd',
  loss='sparse_categorical_crossentropy',
  metrics=[ 'accuracy' ]
)

model.fit(
  X_mnist_train,
  y_mnist_train,
  validation_data=(X_mnist_train, y_mnist_train),
  epochs=10
)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1770 - loss: 336.4047 - val_accuracy: 0.2034 - val_loss: 2.0383
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2024 - loss: 2.0526 - val_accuracy: 0.2075 - val_loss: 2.0407
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2004 - loss: 2.0523 - val_accuracy: 0.2069 - val_loss: 2.0803
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2001 - loss: 2.0708 - val_accuracy: 0.2065 - val_loss: 2.0261
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2024 - loss: 2.0354 - val_accuracy: 0.2052 - val_loss: 2.0193
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.2069 - loss: 2.0265 - val_accuracy: 0.2078 - val_loss: 2.1708
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2021 - loss: 2.0479 - val_accuracy: 0.2039 - val_loss: 2.0407
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2018 - loss: 2.0479 

In [ ]:
import itertools
preds = []
for results in model.predict(X_mnist_test):
  preds.append(np.argmax(results))

comparisons = [a == b for (a, b) in itertools.product(preds, y_mnist_test)]

In [23]:
accuracy = sum(comparisons) / len(comparisons)
print(accuracy)

0.1038914


## Using callable models like layers

Pre-existing models can be called like layers using the Functional API.

1. With the Functional API, trained models can be treated as layers and outputs from a layer can be passed as inputs into these trained models.
2. Able to account for sequenced data (e.g from an image model to a video model). There are wrappers available in the API that utilise the model to predict on every instance in the sequence (e.g `TimeDistributed`)

In [ ]:
# 1. Trained model as a layer
x = Input(shape=(784,))
y = model(x)

In [ ]:
# 2. Turning and image classification to video classification
input_sequence = tf.keras.Input(shape=(10, 28, 28))
processed_sequences = tf.keras.layers.TimeDistributed(model)(input_sequence)

## Creating a model with multiple inputs and outputs

Functional API is able to manage multiple data streams with many input and output layers.

### Example

Predicting the price of a specific house and the elapsed time before its sale

__Inputs__

1. Data about the house such as the nmber of bedrooms, house size, air conditioning, etc.
2. A recent picture of the house

__Outputs__

1. Elapsed time before the sale (categorical - "slow", "fast")
2. Predicted price

In [4]:
# Define the model
## "Model" 1
house_data_inputs = tf.keras.Input(shape=(128,), name='house_data')
x = tf.keras.layers.Dense(64, activation='relu')(house_data_inputs)
block_1_output = tf.keras.layers.Dense(32, activation='relu')(x)

## "Model" 2
house_picture_inputs = tf.keras.Input(shape=(128, 128, 3), name='house_pictures')
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(house_picture_inputs)
x = tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same')(x)
block_2_output = tf.keras.layers.Flatten()(x)

## Combining the outputs
x = tf.keras.layers.concatenate([block_1_output, block_2_output])

## Logistic regression for predicted price
pred_price = tf.keras.layers.Dense(1, activation='relu', name='price')(x)
## Classifier for elapsed time before sale
elapsed_time = tf.keras.layers.Dense(2, activation='softmax', name='elapsed_time')(x)

## Define the final model
model = keras.Model(
  inputs=[house_data_inputs, house_picture_inputs],
  outputs=[pred_price, elapsed_time],
  name='toy_house_pred'
)


In [ ]:
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

## Shared Layers

Some models reuse the same layer multiple times inside the architecture. These layer instances learn features that correspond to multiple paths in the graph of layers. Shared layers are often used to encode inputs from similar spaces.

In the Functional API, to share layers, just instantiate it once and call it on multiple inputs

### Example 

Embedding layer that encodes text information from 2 different inputs with similar vocabulary

In [ ]:
# Variable-length sequence of integers
text_input_a = tf.keras.Input(shape=(None, ), dtype='int32')
text_input_b = tf.keras.Input(shape=(None, ), dtype='int32')

# Embedding layer
shared_embedding = tf.keras.layers.Embedding(1000, 128)

# Reuse the same layer to encode both inputs
encode_input_a = shared_embedding(text_input_a)
encode_input_b = shared_embedding(text_input_b)

## Extracting and reusing nodes in a graph of layers

A node represents the computation that transforms the input to the output within the layer. Multiple nodes are contained within a layer perform the computation.

`tf.keras.application` module contains canned architectures with pre-trained weights.

### A note on Transfer Learning

* Ability to reuse existing architectures or parts of existing architectures in new models
* Improves training phase by decreasing the training time and model performance on related issues
* Used as the starting point since weights are already pre-trained
* Usually used for _weight initialisation_ and _feature extraction_

In [8]:
# resnet model
resnet = tf.keras.applications.resnet.ResNet50()

# display the intermediate layers
intermediate_layers = [layer.output for layer in resnet.layers]
intermediate_layers[:10] # top 10 layers for model

[<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=None, name=keras_tensor_370>,
 <KerasTensor shape=(None, 230, 230, 3), dtype=float32, sparse=False, name=keras_tensor_371>,
 <KerasTensor shape=(None, 112, 112, 64), dtype=float32, sparse=False, name=keras_tensor_372>,
 <KerasTensor shape=(None, 112, 112, 64), dtype=float32, sparse=False, name=keras_tensor_373>,
 <KerasTensor shape=(None, 112, 112, 64), dtype=float32, sparse=False, name=keras_tensor_374>,
 <KerasTensor shape=(None, 114, 114, 64), dtype=float32, sparse=False, name=keras_tensor_375>,
 <KerasTensor shape=(None, 56, 56, 64), dtype=float32, sparse=False, name=keras_tensor_376>,
 <KerasTensor shape=(None, 56, 56, 64), dtype=float32, sparse=False, name=keras_tensor_379>,
 <KerasTensor shape=(None, 56, 56, 64), dtype=float32, sparse=False, name=keras_tensor_380>,
 <KerasTensor shape=(None, 56, 56, 64), dtype=float32, sparse=False, name=keras_tensor_381>]

In [ ]:
feature_layers = intermediate_layers[:-2] # acc to the model architecture, last 2 layers are not feature layers

# Can reuse these feature layers to create a feature extraction model
feature_extraction_model = tf.keras.Model(inputs=resnet.inputs, outputs=feature_layers)

## Summary

* Functional API is more flexible and enables extracting and reusing nodes
* Create non-linear models with multiple inputs and outputs
* Model plotting and whole model saving

### Tips & Tricks

1. Name the layers
2. Separate submodels (in your code)
3. Review the layer summary once model is finished to verify the input and output dimensions between layers
4. Review graph plots to check connection between each layer
5. Use consistent variable names

---

# Keras Subclassing API

Keras is based on object-oriented design principles, so able to create custom models architecture definitions.

More difficult, but more customisation. It's used to build unique architectures and for those that want to have full control over their model.

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "2"

## Creating a custom layer

All layers are subclasses of the `Layer` class and implement these methods:

* `build` - defines weights of layers
* `call` - transformation from inputs to outputs done by layer
* `compute_output_shape` - performs automatic shape inference
* `get_config` & `from_config` - if the layer is serialised and deserialised

In [3]:
# Create a custom layer
class MyCustomDense(tf.keras.layers.Layer):
  # Initialise the class eith the number of units
  def __init__(self, units):
    super(MyCustomDense, self).__init__()
    self.units = units
  
  # Define the weights and bias
  def build(self, input_shape):
    self.w = self.add_weight(
      shape=(input_shape[-1], self.units),
      initializer='random_normal',
      trainable=True
    )
    self.b = self.add_weight(
      shape=(self.units,),
      initializer='random_normal',
      trainable=True
    )
  
  # Applying this layer transformation to the input tensor
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b
  
  # Function to retrieve the configuration
  def get_config(self):
    return {'units': self.units}


In [5]:
# Example
x = tf.ones((2, 2))
my_custom_layer = MyCustomDense(4)
y = my_custom_layer(x)
print(y)

tf.Tensor(
[[-0.00291129 -0.00498378 -0.05527667  0.02332013]
 [-0.00291129 -0.00498378 -0.05527667  0.02332013]], shape=(2, 4), dtype=float32)


In [ ]:
# Using it in a model
inputs = keras.Inputs((12, 4))
outputs = MyCustomDense(2)(inputs)

# Create model
model = model.get_config()

# Reload model from config
new_model = keras.Model.from_config(
  config,
  custom_objects={'MyCustomDense': MyCustomDense}
)

## Creating a custom model

Subclassing the `tf.kears.Model` class to build a fully customisable model.

* `call` - defines the forward pass of the model
* `training` - defines the different behaviour during inference and training

In [7]:
# Load MNIST and greyscale it
mnist = tf.keras.datasets.mnist
(X_mnist_train, y_mnist_train), (X_mnist_test, y_mnist_test) = mnist.load_data()
train_mnist_features = X_mnist_train / 255
test_mnist_features = X_mnist_test / 255

In [6]:
class MyCustomMNIST(tf.keras.Model):
  '''Create the custom layer for the MNIST dataset'''
  def __init__(self, num_classes):
    super(MyCustomMNIST, self).__init__(name='my_mnist_model')
    self.num_classes = num_classes

    # defining the layers
    self.flatten_1 = tf.keras.layers.Flatten()
    self.dropout = tf.keras.layers.Dropout(0.1)
    self.dense_1 = tf.keras.layers.Dense(50, activation='relu')
    self.dense_2 = tf.keras.layers.Dense(10, activation='softmax')
  
  def call(self, inputs, training=False):
    x = self.flatten_1(inputs)
    x = self.dense_1(x)
    # apply the dropout layer if it is training loop
    if training:
      x = self.dropout(x)
    return self.dense_2(x)

In [8]:
my_mnist_model = MyCustomMNIST(10)

In [10]:
my_mnist_model.compile(
  optimizer='sgd',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

my_mnist_model.fit(
  train_mnist_features,
  y_mnist_train,
  validation_data=(test_mnist_features, y_mnist_test),
  epochs=10
)

Epoch 1/10


I0000 00:00:1733877178.913015     153 service.cc:146] XLA service 0x7f92f8005930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733877178.913239     153 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-12-11 00:32:58.947143: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-11 00:32:58.999313: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906


  65/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.1642 - loss: 2.2828

I0000 00:00:1733877179.705877     153 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.6531 - loss: 1.1838 - val_accuracy: 0.8982 - val_loss: 0.3870
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 901us/step - accuracy: 0.8769 - loss: 0.4315 - val_accuracy: 0.9119 - val_loss: 0.3155
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 995us/step - accuracy: 0.8979 - loss: 0.3527 - val_accuracy: 0.9193 - val_loss: 0.2802
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 994us/step - accuracy: 0.9108 - loss: 0.3159 - val_accuracy: 0.9270 - val_loss: 0.2567
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9159 - loss: 0.2944 - val_accuracy: 0.9325 - val_loss: 0.2357
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9219 - loss: 0.2681 - val_accuracy: 0.9369 - val_loss: 0.2220
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9271 - loss: 0.2586 - val_accuracy: 0.9409 - val_loss: 0.2092
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9320 - loss: 0.2397 - val_a

## Summary

* Uses object-oriented design patterns to create the model
* Recommended to only be used if the Functional or Sequential API is unable to fulfill the task

---